In [51]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import os
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction')

from config import *

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import MeanAbsolutePercentageError
import numpy as np


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
# !pip install import-ipynb
%cd {TRAINING_PATH}
# %ls
import import_ipynb
import prepare_data

(X_train, y_train), (X_val, y_val), (X_test, y_test) = prepare_data.load_and_prepare_data(PROCESSED_DATA_PATH,seq_length=20)

print(f'Training set shape: {X_train.shape}')
print(f'Validation set shape: {X_val.shape}')
print(f'Test set shape: {X_test.shape}')
print("Training set last 5 rows:")
print(X_train[-5:])
print("Validation set last 5 rows:")
print(X_val[-5:])
print("Test set last 5 rows:")
print(X_test[-5:])

/content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction/models/lstm/models/training
Training set shape: (17033, 20, 1)
Validation set shape: (3634, 20, 1)
Test set shape: (3635, 20, 1)
Training set last 5 rows:
[[[0.95191467]
  [0.94848814]
  [0.94501285]
  [0.953766  ]
  [0.95628312]
  [0.95540612]
  [0.95472408]
  [0.94983596]
  [0.9574199 ]
  [0.96290889]
  [0.96734226]
  [0.96211306]
  [0.9676183 ]
  [0.96463028]
  [0.9671798 ]
  [0.9693722 ]
  [0.97770311]
  [0.97963562]
  [0.97594921]
  [0.97856375]]

 [[0.94848814]
  [0.94501285]
  [0.953766  ]
  [0.95628312]
  [0.95540612]
  [0.95472408]
  [0.94983596]
  [0.9574199 ]
  [0.96290889]
  [0.96734226]
  [0.96211306]
  [0.9676183 ]
  [0.96463028]
  [0.9671798 ]
  [0.9693722 ]
  [0.97770311]
  [0.97963562]
  [0.97594921]
  [0.97856375]
  [0.98944428]]

 [[0.94501285]
  [0.953766  ]
  [0.95628312]
  [0.95540612]
  [0.95472408]
  [0.94983596]
  [0.9574199 ]
  [0.96290889]
  [0.96734226]
  [0.96211306]
  [0.9676183 ]
  [0.96463028

In [55]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from keras.optimizers import Adam
from keras.metrics import MeanAbsolutePercentageError, RootMeanSquaredError
from keras.regularizers import L1L2

def create_lstm_model(input_shape, dropout_rate=0.2):
    model = Sequential([
        LSTM(64, activation='silu', input_shape=input_shape, return_sequences=True),
        Dropout(dropout_rate),

        LSTM(32, activation='silu', input_shape=input_shape,return_sequences=True),
        Dropout(dropout_rate/2),

        LSTM(16, activation='silu', input_shape=input_shape,return_sequences=True),
        Dropout(dropout_rate/4),

        # Output layer
        Dense(1)
    ])

    # Optimizer
    optimizer = Adam()

    # Metrics
    metrics = [
        MeanAbsolutePercentageError(),
        RootMeanSquaredError()
    ]

    # Compile model
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=metrics
    )

    return model

checkpoint_path = os.path.join(CHECKPOINTS_PATH, 'best_model.keras')
def get_callbacks(model_name, base_path=CHECKPOINTS_PATH):
    # Define file paths
    checkpoint_path = os.path.join(base_path, f'best_{model_name}.keras')

    # Define callbacks
    callbacks = [
        ModelCheckpoint(
            filepath=checkpoint_path,
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=7,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-7,
            verbose=1,
            mode='min'
        )
    ]

    return callbacks


## **Create model**

In [61]:
input_shape = (X_train.shape[1], X_train.shape[2])
model = create_lstm_model(input_shape)
model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_19 (LSTM)                       │ (None, 20, 64)              │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 20, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ (None, 20, 32)              │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 20, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_21 (LSTM)                       │ (None, 20, 16)              │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 20, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 20, 1)               │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 32,465 (126.82 KB)

 Trainable params: 32,465 (126.82 KB)

 Non-trainable params: 0 (0.00 B)

## **Get callbacks and train the model**

In [62]:
callbacks = get_callbacks("lstm-3layer-silu_v1")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=32,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/32
533/533 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0423 - mean_absolute_percentage_error: 14329.9209 - root_mean_squared_error: 0.2055
Epoch 1: val_loss improved from inf to 2.84096, saving model to /content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction/models/lstm/models/checkpoints/best_lstm-3layer-silu_v1.keras
533/533 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - loss: 0.0423 - mean_absolute_percentage_error: 14320.9014 - root_mean_squared_error: 0.2055 - val_loss: 2.8410 - val_mean_absolute_percentage_error: 89.4528 - val_root_mean_squared_error: 1.6855 - learning_rate: 0.0010
Epoch 2/32
532/533 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0404 - mean_absolute_percentage_error: 43610.0391 - root_mean_squared_error: 0.2011
Epoch 2: val_loss improved from 2.84096 to 2.79240, saving model to /content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction/models/lstm/models/checkpoints/best_lstm-3layer-silu_v1.keras
533/533 ━━━━━━━━━━━━━━━━━━━━ 22s 42ms/step - loss: 0.0404 - 

KeyboardInterrupt: 

## **Make predictions**

In [41]:
y_pred = model.predict(X_test)


114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


## **Evaluate the model**

In [46]:
metrics = evaluate_predictions(y_test, y_pred, n_samples=5, scaler=None)
plot_training_history(history)
plot_predictions_bokeh(y_test, y_pred)

Output hidden; open in https://colab.research.google.com to view.

In [42]:
from keras.metrics import MeanAbsolutePercentageError, MeanSquaredError
import numpy as np
import matplotlib.pyplot as plt

def evaluate_predictions(y_test, y_pred, n_samples=5, scaler=None):
    # Flatten arrays if needed
    y_pred = y_pred.flatten()
    y_test = y_test.flatten()

    print(f"Shapes - Predictions: {y_pred.shape}, True Values: {y_test.shape}")

    # Initialize Keras metrics
    mape_metric = MeanAbsolutePercentageError()
    mse_metric = MeanSquaredError()

    # Update metrics
    mape_metric.update_state(y_test, y_pred)
    mse_metric.update_state(y_test, y_pred)

    # Get metric values
    mape = float(mape_metric.result())  # Already in percentage
    mse = float(mse_metric.result())
    rmse = np.sqrt(mse)

    # Calculate MPD
    epsilon = 1e-7
    y_test_safe = np.where(y_test == 0, epsilon, y_test)
    percentage_deviation = np.abs((y_test - y_pred) / y_test_safe) * 100
    mpd_index = np.argmax(percentage_deviation)
    mpd = percentage_deviation[mpd_index]

    # Print metrics
    print("\nModel Performance Metrics:")
    print("-" * 50)
    print(f"MAPE: {mape:.4f}%")
    print(f"MSE: {mse:.8f}")
    print(f"RMSE: {rmse:.8f}")
    print(f"MPD (Maximum Percentage Deviation): {mpd:.4f}%")

    # Print point of maximum deviation
    print(f"\nPoint of Maximum Deviation (Index {mpd_index}):")
    print("-" * 50)
    print(f"True Value: {y_test[mpd_index]:.6f}")
    print(f"Predicted Value: {y_pred[mpd_index]:.6f}")
    print(f"Absolute Difference: {abs(y_test[mpd_index] - y_pred[mpd_index]):.6f}")
    print(f"Percentage Deviation: {percentage_deviation[mpd_index]:.2f}%")

    # Print sample predictions
    print(f"\nFirst {n_samples} Predictions:")
    print("-" * 50)
    print("Index    True Value    Predicted    Difference    % Deviation")
    print("-" * 65)
    for i in range(n_samples):
        diff = y_test[i] - y_pred[i]
        dev = percentage_deviation[i]
        print(f"{i:<8d} {y_test[i]:11.6f}  {y_pred[i]:11.6f}  {diff:11.6f}  {dev:11.2f}%")

    return {
        'mape': mape,
        'mse': mse,
        'rmse': rmse,
        'mpd': mpd,
        'mpd_index': mpd_index,
        'percentage_deviations': percentage_deviation
    }

In [43]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource, HoverTool, Legend

def plot_training_history(history):
    output_notebook()

    # Create data sources
    epochs = list(range(1, len(history.history['loss']) + 1))
    source_loss = ColumnDataSource(data={
        'epoch': epochs,
        'train_loss': history.history['loss'],
        'val_loss': history.history['val_loss']
    })

    source_mape = ColumnDataSource(data={
        'epoch': epochs,
        'train_mape': history.history['mean_absolute_percentage_error'],
        'val_mape': history.history['val_mean_absolute_percentage_error']
    })

    # Create loss plot
    p1 = figure(title='Model Loss Over Time',
               x_axis_label='Epoch',
               y_axis_label='Loss',
               width=600, height=400)

    # Add hover tool
    hover_loss = HoverTool(tooltips=[
        ('Epoch', '@epoch'),
        ('Training Loss', '@train_loss{0.000}'),
        ('Validation Loss', '@val_loss{0.000}')
    ])
    p1.add_tools(hover_loss)

    # Plot loss lines
    l1 = p1.line('epoch', 'train_loss', line_color=Category10[3][0],
                 line_width=2, source=source_loss, legend_label='Training Loss')
    l2 = p1.line('epoch', 'val_loss', line_color=Category10[3][1],
                 line_width=2, source=source_loss, legend_label='Validation Loss')

    # Create MAPE plot
    p2 = figure(title='MAPE Over Time',
                x_axis_label='Epoch',
                y_axis_label='MAPE (%)',
                width=600, height=400)

    # Add hover tool
    hover_mape = HoverTool(tooltips=[
        ('Epoch', '@epoch'),
        ('Training MAPE', '@train_mape{0.00}%'),
        ('Validation MAPE', '@val_mape{0.00}%')
    ])
    p2.add_tools(hover_mape)

    # Plot MAPE lines
    l3 = p2.line('epoch', 'train_mape', line_color=Category10[3][0],
                 line_width=2, source=source_mape, legend_label='Training MAPE')
    l4 = p2.line('epoch', 'val_mape', line_color=Category10[3][1],
                 line_width=2, source=source_mape, legend_label='Validation MAPE')

    # Configure legends
    for p in [p1, p2]:
        p.legend.click_policy = "hide"
        p.legend.location = "top_right"
        p.grid.grid_line_alpha = 0.3

    # Show plots
    show(row(p1, p2))

def plot_predictions_bokeh(y_test, y_pred, n_samples=None):
    output_notebook()

    if n_samples is None:
        n_samples = len(y_test)
    else:
        n_samples = min(n_samples, len(y_test))

    # Prepare data
    x_range = list(range(n_samples))
    source = ColumnDataSource(data={
        'index': x_range,
        'actual': y_test[:n_samples],
        'predicted': y_pred[:n_samples],
        'error': y_test[:n_samples] - y_pred[:n_samples]
    })

    # Create time series plot
    p1 = figure(title='Actual vs Predicted Values',
                x_axis_label='Sample Index',
                y_axis_label='Value',
                width=800, height=400)

    # Add hover tool
    hover = HoverTool(tooltips=[
        ('Index', '@index'),
        ('Actual', '@actual{0.000}'),
        ('Predicted', '@predicted{0.000}'),
        ('Error', '@error{0.000}')
    ])
    p1.add_tools(hover)

    # Plot lines
    l1 = p1.line('index', 'actual', line_color=Category10[3][0],
                 line_width=2, source=source, legend_label='Actual')
    l2 = p1.line('index', 'predicted', line_color=Category10[3][1],
                 line_width=2, source=source, legend_label='Predicted')

    # Configure legend
    p1.legend.click_policy = "hide"
    p1.legend.location = "top_right"
    p1.grid.grid_line_alpha = 0.3

    # Create error distribution plot
    errors = y_test[:n_samples] - y_pred[:n_samples]
    hist, edges = np.histogram(errors, bins=50)

    source_hist = ColumnDataSource(data={
        'top': hist,
        'left': edges[:-1],
        'right': edges[1:]
    })

    p2 = figure(title='Error Distribution',
                x_axis_label='Error',
                y_axis_label='Frequency',
                width=400, height=400)

    p2.quad(top='top', bottom=0, left='left', right='right',
            fill_color=Category10[3][2], line_color="white",
            source=source_hist)

    # Show plots
    show(row(p1, p2))



In [45]:
def calculate_mpd(y_true, y_pred):
    # Convert inputs to numpy arrays if they aren't already
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()

    # Calculate percentage deviations
    epsilon = 1e-7  # Avoid division by zero
    percentage_deviations = np.abs((y_true - y_pred) / (y_true + epsilon)) * 100

    # Find maximum deviation and its index
    max_deviation = np.max(percentage_deviations)
    max_deviation_idx = np.argmax(percentage_deviations)

    return {
        'mpd': max_deviation,
        'index': max_deviation_idx,
        'true_value': y_true[max_deviation_idx],
        'pred_value': y_pred[max_deviation_idx],
        'all_deviations': percentage_deviations
    }

In [64]:
import os
from keras.models import load_model
import pandas as pd

def evaluate_all_models(checkpoint_dir, X_test, y_test, scaler=None):
    # Find all .keras files
    model_files = []
    for root, dirs, files in os.walk(checkpoint_dir):
        for file in files:
            if file.endswith('.keras'):
                model_files.append(os.path.join(root, file))

    print(f"\nFound {len(model_files)} models to evaluate")
    print("=" * 80)

    # Store results
    results = {}

    for model_path in model_files:
        model_name = os.path.basename(model_path).replace('.keras', '')
        print(f"\nEvaluating model: {model_name}")
        print("-" * 80)

        try:
            # Load model
            model = load_model(model_path)

            # Print model summary
            print(f"\nModel Architecture for {model_name}:")
            print("-" * 40)
            model.summary()
            print("-" * 40)

            # Make predictions
            y_pred = model.predict(X_test, verbose=0)

            # Get metrics
            metrics = evaluate_predictions(y_test, y_pred, n_samples=5, scaler=scaler)

            # Store results
            results[model_name] = {
                'metrics': metrics,
                'model': model
            }

            # Print metrics
            print(f"\nMetrics for {model_name}:")
            print("-" * 40)
            print(f"MAPE: {metrics['mape']:.4f}%")
            print(f"RMSE: {metrics['rmse']:.4f}")
            print(f"MPD: {metrics['mpd']:.4f}%")
            print("-" * 40)

        except Exception as e:
            print(f"Error evaluating {model_name}: {str(e)}")

    # Create comparison DataFrame
    metrics_df = pd.DataFrame({
        model_name: {
            'MAPE (%)': results[model_name]['metrics']['mape'],
            'RMSE': results[model_name]['metrics']['rmse'],
            'MPD (%)': results[model_name]['metrics']['mpd']
        }
        for model_name in results.keys()
    }).T

    print("\nModel Comparison Summary:")
    print("=" * 80)
    print(metrics_df.to_string())
    print("=" * 80)

    return results, metrics_df


# Evaluate all models
results, metrics_df = evaluate_all_models(CHECKPOINTS_PATH, X_test, y_test)


Found 9 models to evaluate

Evaluating model: best_lstm-without-batchnormalization
--------------------------------------------------------------------------------

Model Architecture for best_lstm-without-batchnormalization:
----------------------------------------


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_32 (LSTM)                       │ (None, 14, 64)              │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_40 (Dropout)                 │ (None, 14, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_33 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_41 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_42 (Dropout)                 │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 89,573 (349.90 KB)

 Trainable params: 29,857 (116.63 KB)

...

 Total params: 89,701 (350.40 KB)

 Trainable params: 29,889 (116.75 KB)

 Non-trainable params: 32 (128.00 B)

 Optimizer params: 59,780 (233.52 KB)

----------------------------------------
Shapes - Predictions: (3635,), True Values: (3635,)

Model Performance Metrics:
--------------------------------------------------
MAPE: 2.0063%
MSE: 0.04800412
RMSE: 0.21909842
MPD (Maximum Percentage Deviation): 22.0029%

Point of Maximum Deviation (Index 2431):
--------------------------------------------------
True Value: 3.867826
Predicted Value: 4.718860
Absolute Difference: 0.851034
Percentage Deviation: 22.00%

First 5 Predictions:
--------------------------------------------------
Index    True Value    Predicted    Difference    % Deviation
-----------------------------------------------------------------
0           1.732518     1.752554    -0.020036         1.16%
1           1.752379     1.752313     0.000066         0.00%
2           1.729822     1.749508    -0.019686         1.14%
3           1.768927     1.749462     0.019465         1.10%
4           1.783527     1.747116     0.036411         2.04%

Metrics for best_model:
------